In [7]:
import parselmouth
from parselmouth.praat import call, run_file
import glob
import pandas as pd
import numpy as np
import scipy
from scipy.stats import binom
from scipy.stats import ks_2samp
from scipy.stats import ttest_ind
import os



In [9]:
def run_praat_file(m, p):
    """
    p : path to dataset folder
    m : path to file
    returns : objects outputed by the praat script
    """
    sound=p+"/"+"dataset"+"/"+"audioFiles"+"/"+m+".wav"
#     sourcerun=p+"/"+"dataset"+"/"+"essen"+"/"+"myspsolution.praat"
    sourcerun = r"/home/ubuntu/speechRecognition/my_prosody/myprosodyGit/myprosody/dataset/essen/myspsolution.praat"
    path=p+"/"+"dataset"+"/"+"audioFiles"+"/"

#     assert os.path.isfile(sound), "Wrong path to audio file"
#     assert os.path.isfile(sourcerun), "Wrong path to praat script"
#     assert os.path.isdir(path), "Wrong path to audio files"

#     try:
    objects= run_file(sourcerun, -20, 2, 0.3, "yes",sound,path, 80, 400, 0.01, capture_output=True)
    print (objects[0]) # This will print the info from the sound object, and objects[0] is a parselmouth.Sound object
    z1=str( objects[1]) # This will print the info from the textgrid object, and objects[1] is a parselmouth.Data object with a TextGrid inside
    z2=z1.strip().split()
    return z2
#     except:
#         z3 = 0
#         print ("Try again the sound of the audio was not clear")
        
        
def myspsr(m,p):
    """
    Measure the rate of speech (speed)
    """
    z2 = run_praat_file(m, p)
    z3=int(z2[2]) # will be the integer number 10
    z4=float(z2[3]) # will be the floating point number 8.3
    print ("rate_of_speech=",z3,"# syllables/sec original duration")
    return z3

m = "audioFiles/suun3"
p = "/home/ubuntu/speechRecognition/my_prosody/myprosodyGit/myprosody"

myspsr(m,p)

PraatError: Cannot open file “/home/ubuntu/speechRecognition/my_prosody/myprosodyGit/myprosody/dataset/audioFiles/audioFiles/suun3.wav”.
Script line 46 not performed or completed:
« Read from file... 'soundin$' »
Script not completed.

In [14]:
import inspect
lines = inspect.getsource(run_file)
print(lines)

TypeError: module, class, method, function, traceback, frame, or code object was expected, got builtin_function_or_method

In [18]:
sourcerun = r"/home/ubuntu/speechRecognition/my_prosody/myprosodyGit/myprosody/dataset/essen/myspsolution.praat"
path=p+"/"+"dataset"+"/"+"audioFiles"+"/"
sound=p+"/"+"dataset"+"/"+"audioFiles"+"/"+m+".wav"

print(path)
print(sound)
objects= run_file(sourcerun, -20, 2, 0.3, "yes",sound,path, 80, 400, 0.01, capture_output=True)

/home/ubuntu/speechRecognition/my_prosody/myprosodyGit/myprosody/dataset/audioFiles/
/home/ubuntu/speechRecognition/my_prosody/myprosodyGit/myprosody/dataset/audioFiles/audioFiles/suun3.wav


PraatError: Cannot open file “/home/ubuntu/speechRecognition/my_prosody/myprosodyGit/myprosody/dataset/audioFiles/audioFiles/suun3.wav”.
Script line 46 not performed or completed:
« Read from file... 'soundin$' »
Script not completed.

In [71]:
import math
import pandas as pd
import parselmouth

from glob import glob
from parselmouth.praat import call


def speech_rate(filename):
    silencedb = -25
    mindip = 2
    minpause = 0.3
    sound = parselmouth.Sound(filename)
    originaldur = sound.get_total_duration()
    intensity = sound.to_intensity(50)
    start = call(intensity, "Get time from frame number", 1)
    nframes = call(intensity, "Get number of frames")
    end = call(intensity, "Get time from frame number", nframes)
    min_intensity = call(intensity, "Get minimum", 0, 0, "Parabolic")
    max_intensity = call(intensity, "Get maximum", 0, 0, "Parabolic")

    # get .99 quantile to get maximum (without influence of non-speech sound bursts)
    max_99_intensity = call(intensity, "Get quantile", 0, 0, 0.99)

    # estimate Intensity threshold
    threshold = max_99_intensity + silencedb
    threshold2 = max_intensity - max_99_intensity
    threshold3 = silencedb - threshold2
    if threshold < min_intensity:
        threshold = min_intensity

    # get pauses (silences) and speakingtime
    textgrid = call(intensity, "To TextGrid (silences)", threshold3, minpause, 0.1, "silent", "sounding")
    silencetier = call(textgrid, "Extract tier", 1)
    silencetable = call(silencetier, "Down to TableOfReal", "sounding")
    npauses = call(silencetable, "Get number of rows")
    speakingtot = 0
    for ipause in range(npauses):
        pause = ipause + 1
        beginsound = call(silencetable, "Get value", pause, 1)
        endsound = call(silencetable, "Get value", pause, 2)
        speakingdur = endsound - beginsound
        speakingtot += speakingdur

    intensity_matrix = call(intensity, "Down to Matrix")
    # sndintid = sound_from_intensity_matrix
    sound_from_intensity_matrix = call(intensity_matrix, "To Sound (slice)", 1)
    # use total duration, not end time, to find out duration of intdur (intensity_duration)
    # in order to allow nonzero starting times.
    intensity_duration = call(sound_from_intensity_matrix, "Get total duration")
    intensity_max = call(sound_from_intensity_matrix, "Get maximum", 0, 0, "Parabolic")
    point_process = call(sound_from_intensity_matrix, "To PointProcess (extrema)", "Left", "yes", "no", "Sinc70")
    # estimate peak positions (all peaks)
    numpeaks = call(point_process, "Get number of points")
    t = [call(point_process, "Get time from index", i + 1) for i in range(numpeaks)]

    # fill array with intensity values
    timepeaks = []
    peakcount = 0
    intensities = []
    for i in range(numpeaks):
        value = call(sound_from_intensity_matrix, "Get value at time", t[i], "Cubic")
        if value > threshold:
            peakcount += 1
            intensities.append(value)
            timepeaks.append(t[i])

    # fill array with valid peaks: only intensity values if preceding
    # dip in intensity is greater than mindip
    validpeakcount = 0
    currenttime = timepeaks[0]
    currentint = intensities[0]
    validtime = []

    for p in range(peakcount - 1):
        following = p + 1
        followingtime = timepeaks[p + 1]
        dip = call(intensity, "Get minimum", currenttime, timepeaks[p + 1], "None")
        diffint = abs(currentint - dip)
        if diffint > mindip:
            validpeakcount += 1
            validtime.append(timepeaks[p])
        currenttime = timepeaks[following]
        currentint = call(intensity, "Get value at time", timepeaks[following], "Cubic")

    # Look for only voiced parts
    pitch = sound.to_pitch_ac(0.02, 30, 4, False, 0.03, 0.25, 0.01, 0.35, 0.25, 450)
    voicedcount = 0
    voicedpeak = []

    for time in range(validpeakcount):
        querytime = validtime[time]
        whichinterval = call(textgrid, "Get interval at time", 1, querytime)
        whichlabel = call(textgrid, "Get label of interval", 1, whichinterval)
        value = pitch.get_value_at_time(querytime) 
        if not math.isnan(value):
            if whichlabel == "sounding":
                voicedcount += 1
                voicedpeak.append(validtime[time])

    # calculate time correction due to shift in time for Sound object versus
    # intensity object
    timecorrection = originaldur / intensity_duration

    # Insert voiced peaks in TextGrid
    call(textgrid, "Insert point tier", 1, "syllables")
    for i in range(len(voicedpeak)):
        position = (voicedpeak[i] * timecorrection)
        call(textgrid, "Insert point", 1, position, "")

    # return results
    speakingrate = voicedcount / originaldur
    articulationrate = voicedcount / speakingtot
    npause = npauses - 1
    asd = speakingtot / voicedcount
    speechrate_dictionary = {'soundname':file,
                             'nsyll':voicedcount,
                             'npause': npause,
                             'dur(s)':originaldur,
                             'phonationtime(s)':intensity_duration,
                             'speechrate(nsyll / dur)': speakingrate,
                             "articulation rate(nsyll / phonationtime)":articulationrate,
                             "ASD(speakingtime / nsyll)":asd}
    return speechrate_dictionary


def get_files():
#     files = glob('test_voices/*.wav')
#     files.extend(glob('test_voices/*.mp3'))
#     files.extend(glob('test_voices/*.ogg'))
#     files.extend(glob('test_voices/*.aiff'))
#     files.extend(glob('test_voices/*.aifc'))
#     files.extend(glob('test_voices/*.au'))
#     files.extend(glob('test_voices/*.nist'))
#     files.extend(glob('test_voices/*.flac'))
    files = ["/home/ubuntu/emotional_sounds/wav/Varshit.wav"]
    return files


# if __name__ == "__main__":
files = get_files()
cols = ['soundname', 'nsyll', 'npause', 'dur(s)', 'phonationtime(s)', 'speechrate(nsyll / dur)', 'articulation '
        'rate(nsyll / phonationtime)', 'ASD(speakingtime / nsyll)']
datalist = []
for file in files:
    import time
    t1 = time.time()
    speechrate_dictionary = speech_rate(file)
    t2 = time.time()
    print(t2-t1)
    datalist.append(speechrate_dictionary)
df = pd.DataFrame(datalist)
df

23.6434063911438


,soundname,nsyll,npause,dur(s),phonationtime(s),speechrate(nsyll / dur),articulation rate(nsyll / phonationtime),ASD(speakingtime / nsyll)
0,/home/ubuntu/emotional_sounds/wav/Varshit.wav,2197,248,688.670375,688.670375,3.190205,4.618341,0.216528


In [94]:
import IPython
IPython.display.Audio("/home/ubuntu/speechRecognition/vosk-api/python/example/submission_audios/question_submission_79.wav")

In [31]:
df.T.to_dict(orient = 'index')

{'soundname': {0: '/home/ubuntu/pause_audios/wav/podcast_17021.wav'},
 'nsyll': {0: 1122},
 'npause': {0: 88},
 'dur(s)': {0: 288.29895691609977},
 'phonationtime(s)': {0: 288.29895691609977},
 'speechrate(nsyll / dur)': {0: 3.8917934771665594},
 'articulation rate(nsyll / phonationtime)': {0: 4.578144050849839},
 'ASD(speakingtime / nsyll)': {0: 0.21842912518542756}}